<a href="https://colab.research.google.com/github/PavelNovikov888/hiperpameters_optimization/blob/master/%D0%9F%D0%BE%D0%B4%D0%B1%D0%BE%D1%80_%D0%B3%D0%B8%D0%BF%D0%B5%D1%80%D0%BF%D0%B0%D1%80%D0%B0%D0%BC%D0%B5%D1%82%D1%80%D0%BE%D0%B2_%D0%91%D0%B0%D0%B9%D0%B5%D1%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ЭТАП 0. ПОДГОТОВКА



In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

from sklearn.model_selection import GridSearchCV, KFold, RandomizedSearchCV
from sklearn.model_selection import cross_val_score

import numpy as np

# генерация данных
from sklearn.datasets import make_classification

# для настройки гиперпараметров
from scipy.stats import loguniform
from sklearn.pipeline import Pipeline

## Генерация данных

С помощью make_classification из sklearn.datasets сгенерируем датасет с 1 000 объектов, 10 признаками и бинарным таргетом.

In [2]:
# from sklearn.datasets import make_classification
# создадим случайную задачу регрессии
X, y = make_classification(
    # количество наблюдений
    n_samples=1000,
    # количество признаков
    n_features=10,
    # количество признаков, использованных для построения линейной модели, используемой для генерации выходных данных.
    n_classes = 2,
    random_state=42,
)

In [3]:
X.shape

(1000, 10)

In [4]:
y[:10]

array([0, 1, 1, 0, 1, 0, 0, 1, 1, 0])

## Создадим сетку гиперпараметров

In [5]:
search_space = {
                'lr__penalty' : ['l1', 'l2'], # способ регуляризации
                # loguniform.rvs - равномерно распределённые случайные величины
                'lr__C' : loguniform.rvs(10**(-4),10**2, size=100) # коэффицент регуляризациии
                }

In [6]:
search_space['lr__C'][:10]

array([9.94537269e-01, 9.59970017e+00, 1.75121924e+01, 7.23387374e-02,
       3.33073195e+01, 2.64268734e+00, 5.97463675e-02, 1.21001897e-03,
       2.70354099e+00, 8.47289525e-02])

##  Инициализируем модель в виде пайплайна

Параметр penalty будем выбирать равновероятно из  **['l1', 'l2']**, а параметр регуляризации С из лог-равномерного распределения может принимать значения **[10-4, 102]**.

In [7]:
model = Pipeline([('lr', LogisticRegression(random_state=42,
                            solver='liblinear'))])

# GRID SEARCH

Перебор по сетке будем выполнять с помощью GridSearchCV из sklearn.model_selection.

При его инициализации укажем несколько параметров:

- Модель (пайплайн)  
- Сетку с параметрами
- Количество разбиений для (Stratified)KFold, который используется по умолчанию = 3
- Метрику для оценки производительности модели с перекрёстной проверкой на тестовом наборе scoring='accuracy'.

In [8]:
cv = KFold(3)
grid_search = GridSearchCV(model, param_grid=search_space, cv = cv, scoring='accuracy')
grid_search

GridSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('lr',
                                        LogisticRegression(random_state=42,
                                                           solver='liblinear'))]),
             param_grid={'lr__C': array([9.94537269e-01, 9.59970017e+00, 1.75121924e+01, 7.23387374e-02,
       3.33073195e+01, 2.64268734e+00, 5.97463675e-02, 1.21001897e-03,
       2.70354099e+00, 8.47289525e-02, 3.21603731e+00, 1....
       4.26008183e+00, 2.78051885e+00, 2.63958130e-04, 4.35266886e-02,
       9.94353675e-04, 1.46407226e+00, 2.12590021e-02, 4.13646831e-01,
       1.39444213e-04, 4.26428314e-01, 8.80746953e-02, 8.75296540e+00,
       7.51750147e-01, 3.79923951e-02, 2.54776870e-03, 5.01242360e-03,
       1.04075881e-02, 1.80599331e-01, 8.27564815e-04, 7.45451170e+01,
       3.24632332e-01, 3.46030731e-01, 1.35957464e-02, 1.51036134e-04]),
                         'lr__penalty': ['l1', 'l2']},
             scoring='accuracy')

Обучим GridSearchCV с параметрами

In [9]:
model_grid = grid_search.fit(X, y)

Теперь можем посмотреть на лучший score и наилучшие гиперпараметры:

In [10]:
print(model_grid.best_score_)
print(model_grid.best_params_)

0.8680027332721943
{'lr__C': 0.002623872310270619, 'lr__penalty': 'l2'}


# RANDOM SEARCH

Для случайного поиска воспользуемся RandomizedSearchCV из sklearn.model_selection.  
Помимо модели, параметров, скоринга и cv, зададим n_iter.   
Он отвечает за количество выбранных комбинаций параметров.   
Чем больше n_iter, тем дольше будет работать поиск.   
Соответственно, максимально возможный n_iter приближает RandomizedSearchCV к GridSearchCV.

Зададим n_iter = 70 и инициализируем RandomizedSearchCV.

In [11]:
cv = KFold(3)
random_search = RandomizedSearchCV(model, param_distributions=search_space, cv = cv, n_iter = 70, scoring='accuracy')
random_search

RandomizedSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=False),
                   estimator=Pipeline(steps=[('lr',
                                              LogisticRegression(random_state=42,
                                                                 solver='liblinear'))]),
                   n_iter=70,
                   param_distributions={'lr__C': array([9.94537269e-01, 9.59970017e+00, 1.75121924e+01, 7.23387374e-02,
       3.33073195e+01, 2.64268734e+00, 5.97463675e-02, 1.21001897e-03,
       2.70354099e+00, 8.47289...
       4.26008183e+00, 2.78051885e+00, 2.63958130e-04, 4.35266886e-02,
       9.94353675e-04, 1.46407226e+00, 2.12590021e-02, 4.13646831e-01,
       1.39444213e-04, 4.26428314e-01, 8.80746953e-02, 8.75296540e+00,
       7.51750147e-01, 3.79923951e-02, 2.54776870e-03, 5.01242360e-03,
       1.04075881e-02, 1.80599331e-01, 8.27564815e-04, 7.45451170e+01,
       3.24632332e-01, 3.46030731e-01, 1.35957464e-02, 1.51036134e-04]),
                                        'lr__penalty': ['l1', 'l2']},
                   scoring='accuracy')

Обучим RandomSearchCV с параметрами

In [12]:
model_grid_r = random_search.fit(X, y)

Теперь можем посмотреть на лучший score и наилучшие гиперпараметры:

In [13]:
print(model_grid_r.best_score_)
print(model_grid_r.best_params_)

0.8680027332721943
{'lr__penalty': 'l2', 'lr__C': 0.0026180648711200847}


# Генетический алгоритм ТРОТ


In [14]:
!pip install tpot
from tpot import TPOTClassifier

Зададим параметры немного иначе, при этом оставив те же значения.

In [15]:
search_space = {
                'penalty' : ['l1', 'l2'],
                'C' : loguniform.rvs(10**(-4),10**2, size=100)
                }

Перед инициализацией классификатора добавим следующие параметры

In [16]:
cv = KFold(3)
tpot_classifier = TPOTClassifier(cv = cv,
                                scoring='accuracy',
                                generations = 5, # количество поколений в процессе оптимизации
                                population_size = 50, # число особей, сохраняемых в популяции генетического программирования в каждом поколении
                                offspring_size = 25, # количество потомства, которое нужно произвести в каждом поколении генетического программирования
                                verbosity = 2,
                                config_dict = {'sklearn.linear_model.LogisticRegression': search_space}, # cловарь с гиперпараметрами для оптимизации для выбранной модели)
)
tpot_classifier

TPOTClassifier(config_dict={'sklearn.linear_model.LogisticRegression': {'C': array([1.16456477e-04, 1.54881043e-01, 5.71743302e-04, 1.81914893e-03,
       1.05389084e-04, 7.18240482e-03, 2.75567557e-01, 1.66775518e-02,
       6.07988985e-02, 2.49301261e-02, 2.24077437e-02, 9.07240559e+00,
       3.31852105e-04, 1.73495938e-04, 3.92837326e-04, 4.89333338e+01,
       5.04452410e-02, 4.39024978e+0...
       1.06143296e-01, 3.80614236e+01, 2.82708082e-04, 8.48504567e-01,
       3.74997145e+00, 8.46624230e-02, 1.35913263e-03, 9.26872778e-03,
       1.82484706e-01, 9.01592933e-02, 3.78321850e-04, 2.88336713e-04,
       3.02762558e+01, 1.14086437e+00, 6.19699082e+00, 1.27197235e-02]),
                                                                        'penalty': ['l1',
                                                                                    'l2']}},
               cv=KFold(n_splits=3, random_state=None, shuffle=False),
               generations=5, offspring_size=25, population_size=50,
               scoring='accuracy', verbosity=2)

Обучим инициализированный классификатор

In [17]:
tpot_classifier.fit(X, y)

Optimization Progress:   0%|          | 0/175 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.8680087272901643

Generation 2 - Current best internal CV score: 0.8680087272901643

Generation 3 - Current best internal CV score: 0.8680087272901643

Generation 4 - Current best internal CV score: 0.8680087272901643

Generation 5 - Current best internal CV score: 0.8680087272901643

Best pipeline: LogisticRegression(LogisticRegression(input_matrix, C=0.0011655717739836615, penalty=l2), C=0.15107262864979903, penalty=l2)


TPOTClassifier(config_dict={'sklearn.linear_model.LogisticRegression': {'C': array([1.16456477e-04, 1.54881043e-01, 5.71743302e-04, 1.81914893e-03,
       1.05389084e-04, 7.18240482e-03, 2.75567557e-01, 1.66775518e-02,
       6.07988985e-02, 2.49301261e-02, 2.24077437e-02, 9.07240559e+00,
       3.31852105e-04, 1.73495938e-04, 3.92837326e-04, 4.89333338e+01,
       5.04452410e-02, 4.39024978e+0...
       1.06143296e-01, 3.80614236e+01, 2.82708082e-04, 8.48504567e-01,
       3.74997145e+00, 8.46624230e-02, 1.35913263e-03, 9.26872778e-03,
       1.82484706e-01, 9.01592933e-02, 3.78321850e-04, 2.88336713e-04,
       3.02762558e+01, 1.14086437e+00, 6.19699082e+00, 1.27197235e-02]),
                                                                        'penalty': ['l1',
                                                                                    'l2']}},
               cv=KFold(n_splits=3, random_state=None, shuffle=False),
               generations=5, offspring_size=25, population_size=50,
               scoring='accuracy', verbosity=2)

Для извлечения наилучших параметров воспользуемся вспомогательным кодом

In [18]:
args = {}
for arg in tpot_classifier._optimized_pipeline:
    if type(arg) != 'Primitive':
        try:
            if arg.value.split('__')[1].split('=')[0] in ['C', 'penalty']:
                args[arg.value.split('__')[1].split('=')[0]] = (arg.value.split('__')[1].split('=')[1])
            else:
                args[arg.value.split('__')[1].split('=')[0]] = float(arg.value.split('__')[1].split('=')[1])
        except:
            pass
params = args

In [19]:
tpot_classifier._optimized_pipeline

In [20]:
print(params)

{'C': '0.15107262864979903', 'penalty': 'l2'}


# БАЙЕСОВСКАЯ ОПТИМИЗАЦИЯ

Здесь значения гиперпараметров в текущей итерации выбираются с учётом результатов на предыдущем шаге.

Основная идея алгоритма заключается в следующем: на каждой итерации подбора находится компромисс между исследованием регионов с самыми удачными из найденных комбинаций гиперпараметров и исследованием регионов с большой неопределённостью (где могут находиться ещё более удачные комбинации).   
Это позволяет во многих случаях найти лучшие значения параметров модели за меньшее количество времени.

Мы рассмотрим библиотеку **Hyperopt** для подбора гиперпарметров.   
В ней реализовано три алгоритма оптимизации:

- классический Random Search;
- метод байесовской оптимизации Tree of Parzen Estimators (TPE);
- Simulated Annealing, метод имитации отжига.

**Hyperopt** может работать с разными типами гиперпараметров — непрерывными, дискретными, категориальными и так далее, что является важным преимуществом этой библиотеки.



In [21]:
!pip install hyperopt

In [22]:
from functools import partial
from sklearn.model_selection import StratifiedKFold
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
# fmin - основная функция, она будет минимизировать наш функционал
# tpe - алгоритм оптимизации
# hp - включает набор методов для объявления пространства поиска гиперпараметров
# trails - используется для логирования результатов

functools.partial(func, *args, **keywords) - возвращает partial-объект (по сути, функцию), который при вызове вызывается как функция func, но дополнительно передают туда позиционные аргументы args, и именованные аргументы kwargs. Если другие аргументы передаются при вызове функции, то позиционные добавляются в конец, а именованные расширяют и перезаписывают.


In [23]:
# Например:
from functools import partial
basetwo = partial(int, base=2)
basetwo.__doc__ = 'Convert base 2 string to an int.' #Преобразование строки по основанию 2 в int
print(basetwo('10010'))
print(basetwo('10011'))
print(basetwo('100'))

18
19
4


Укажем объект для сохранения истории поиска (Trials).   
Это очень удобно, поскольку можно сохранять, прерывать и затем продолжать процесс поиска гиперпараметров.   


In [24]:
trials = Trials() # используется для логирования результатов

Запустим сам процесс подбора с помощью функции fmin.  
Укажем в качестве алгоритма поиска tpe.suggest — байесовскую оптимизацию.   
Для Random Search нужно указать tpe.rand.suggest.

Нам понадобится воспользоваться вспомогательной функцией, которую мы будем оптимизировать:

In [25]:
def objective(params, model,  X_train, y_train):
    """
    Кросс-валидация с текущими гиперпараметрами

    :params: гиперпараметры
    :pipeline: модель
    :X_train: матрица признаков
    :y_train: вектор меток объектов
    :return: средняя точность на кросс-валидации
    """

    # задаём модели требуемые параметры
    model.set_params(**params)

    # задаём параметры кросс-валидации (стратифицированная 4-фолдовая с перемешиванием)
    skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=1)

    # проводим кросс-валидацию
    score = cross_val_score(estimator=model, X=X_train, y=y_train,
                            scoring='accuracy', cv=skf, n_jobs=-1)

    # возвращаем результаты, которые записываются в Trials()
    return   {'loss': -score.mean(), 'params': params, 'status': STATUS_OK}

Задать гиперпараметры с использованием распределения из hyperopt:

In [27]:
search_space = {
                'lr__penalty' : hp.choice(label='penalty',
                          options=['l1', 'l2']),
                'lr__C' : hp.loguniform(label='C',
                        low=-4*np.log(10),
                        high=2*np.log(10))
}

In [32]:
trials = Trials()
best = fmin(
          # функция для оптимизации
            fn=partial(objective, model=model, X_train=X, y_train=y),
          # пространство поиска гиперпараметров
            space=search_space,
          # алгоритм поиска
            algo=tpe.suggest,
          # число итераций (можно ещё указать  время поиска)
            max_evals=40,
          # куда сохранять историю поиска
            trials=trials,
          # random state
            rstate=np.random.default_rng(42),
          # progressbar
            show_progressbar=True
        )

100%|██████████| 40/40 [00:02<00:00, 18.49trial/s, best loss: -0.865]


In [33]:
print("Наилучшие значения гиперпараметров {}".format(best))

Наилучшие значения гиперпараметров {'C': 0.0047921009992456555, 'penalty': 1}


Результат у Байесовской оптимизации наилучший.
score = -0.865